# Day 2

Today, we will start using nf-core pipelines to find differentially abundant genes in our dataset. 
We are using data from the following paper: https://www.nature.com/articles/s41593-023-01350-3#Sec10

1. Please take some time to read through the paper and understand their approach, hypotheses and goals.

What was the objective of the study?


The objective of the study was to investigate transcriptomic changes associated with chronic opioid exposure and physical dependency on opioids while suffering from chronic neuropathic pain.  

Pryce, Kerri D., et al. "Oxycodone withdrawal induces HDAC1/HDAC2-dependent transcriptional maladaptations in the reward pathway in a mouse model of peripheral nerve injury." Nature neuroscience 26.7 (2023): 1229-1244.

What do the conditions mean?

oxy: oxycodone exposure <br>
sal: saline exposure

Pryce, Kerri D., et al. "Oxycodone withdrawal induces HDAC1/HDAC2-dependent transcriptional maladaptations in the reward pathway in a mouse model of peripheral nerve injury." Nature neuroscience 26.7 (2023): 1229-1244.

What do the genotypes mean?

SNI: spared nerve injury


Sham: control group that undergoes an inactive procedure to simulate the active procedure of the other group 

Pryce, Kerri D., et al. "Oxycodone withdrawal induces HDAC1/HDAC2-dependent transcriptional maladaptations in the reward pathway in a mouse model of peripheral nerve injury." Nature neuroscience 26.7 (2023): 1229-1244.
https://www.cancer.gov/publications/dictionaries/cancer-terms/def/sham-controlled (20.10.2024)

Imagine you are the bioinformatician in the group who conducted this study. They hand you the raw files and ask you to analyze them.

What would you do?<br>
I would do a differential abundance analysis.

Which groups would you compare to each other?<br>
I would compare the sham control group with the SNI control, SNI treated and the Sham treated group.

Please also mention which outcome you would expect to see from each comparison. <br>
I would expect similar outcomes to the paper. Thus for the comparison of the Sham control group with the Sham treated group I would expect the most differentially expressed genes, for the comparison of SNI control versus Sham control the second most and for the comparison of SNI treated versus Sham control the fewest.

Your group gave you a very suboptimal excel sheet (conditions_runs_oxy_project.xlsx) to get the information you need for each run they uploaded to the SRA.<br>
So, instead of directly diving into downloading the data and starting the analysis, you first need to sort the lazy table.<br>
Use Python and Pandas to get the table into a more sensible order.<br>
Then, perform some overview analysis and plot the results
1. How many samples do you have per condition?
2. How many samples do you have per genotype?
3. How often do you have each condition per genotype?

In [1]:
import pandas as pd
import numpy as np 

con = pd.read_excel('conditions_runs_oxy_project.xlsx')

print('Number of samples with Sal condition: {}'.format(con.count()['condition: Sal']))
print('Number of samples with Oxy condition: {}'.format(con.count()['Condition: Oxy']))

Number of samples with Sal condition: 8
Number of samples with Oxy condition: 8


In [2]:

print('Number of samples with SNI genotype: {}'.format(con.count()['Genotype: SNI']))
print('Number of samples with Sham genotype: {}'.format(con.count()['Genotype: Sham']))

Number of samples with SNI genotype: 8
Number of samples with Sham genotype: 8


In [3]:
print('Number of Sal conditions per SNI genotype: {} '.format(len(con[(con['Genotype: SNI']=='x') & (con['condition: Sal']=='x')])))
print('Number of Oxy conditions per SNI genotype: {} '.format(len(con[(con['Genotype: SNI']=='x') & (con['Condition: Oxy']=='x')])))
print('Number of Sal conditions per Sham genotype: {} '.format(len(con[(con['Genotype: Sham']=='x') & (con['condition: Sal']=='x')])))
print('Number of Oxy conditions per Sham genotype: {} '.format(len(con[(con['Genotype: Sham']=='x') & (con['Condition: Oxy']=='x')])))

Number of Sal conditions per SNI genotype: 4 
Number of Oxy conditions per SNI genotype: 4 
Number of Sal conditions per Sham genotype: 4 
Number of Oxy conditions per Sham genotype: 4 


They were so kind to also provide you with the information of the number of bases per run, so that you can know how much space the data will take on your Cluster.<br>
Add a new column to your fancy table with this information (base_counts.csv) and sort your dataframe according to this information and the condition.

Then select the 2 smallest runs from your dataset and download them from SRA (maybe an nf-core pipeline can help here?...)

In [4]:
bc = pd.read_csv('base_counts.csv',delimiter = ',')
con_bc = con.merge(bc,left_on='Run',right_on='Run')
con_bc = con_bc.sort_values('Bases')

In [5]:
con_bc = con_bc.sort_values('Bases')
run_ids = con_bc.head(2)
run_ids = list(run_ids['Run'])

with open('samplesheet.csv', 'w') as f:
    for run_id in run_ids:
        f.write(run_id+'\n')

In [12]:
!nextflow run nf-core/fetchngs -profile docker --input samplesheet.csv --outdir DAY2_1 -resume --max_memory '7GB' --max_cpus 4

[sudo] Passwort für jana: 
 N E X T F L O W   ~  version 24.04.4

Launching `https://github.com/nf-core/fetchngs` [admiring_kowalevski] DSL2 - revision: 8ec2d934f9 [master]

WARN: Access to undefined parameter `monochromeLogs` -- Initialise it to a default value eg. `params.monochromeLogs = some_value`


------------------------------------------------------
                                        ,--./,-.
        ___     __   __   __   ___     /,-._.--~'
  |\ | |__  __ /  ` /  \ |__) |__         }  {
  | \| |       \__, \__/ |  \ |___     \`-._,-`-,
                                        `._,._,'
  nf-core/fetchngs v1.12.0-g8ec2d93
------------------------------------------------------
Core Nextflow options
  revision       : master
  runName        : admiring_kowalevski
  containerEngine: docker
  launchDir      : /home/jana/UNI/Master/IISemester/compworkflows/day2/day2
  workDir        : /home/jana/UNI/Master/IISemester/compworkflows/day2/day2/work
  projectDir     : /root/.nextflo

While your files are downloading, get back to the paper and explain how you would try to reproduce the analysis.<br>
When you are done with this shout, so we can discuss the different ideas.

To try to reproduce the result of the differential expression analysis I would use the pipeline [nf-core/rnaseq](https://nf-co.re/rnaseq/3.16.1/). I would compare the same groups as they did in the paper (SNI-Oxy versus Sham-Sal, SNI-Sal versus Sham-Sal, Sham-Oxy versus Sham-Sal). However, it is likely that the results will deviate from the paper, since they do not employ a nf-core pipeline or a similar framework, which ensures reproducibility.